1. Implement two classification algorithms for detecting faces and classifying digits:
(a) Perceptron
(b) Two-layer Neural Network (input layer, one hidden layer, output)
2. Design the features for each of the two problems, and write a program for extracting the features from each image.
You can also use the raw pixels directly as features (and avoid the process of designing features altogether) if that
results in a better classification accuracy.
3. Train the two algorithms on the part of the data set that is reserved for training. First, use only 10% of the data points
that are reserved for training, then 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%, and finally 100%. All the results
should a function of the number of data points used for training.
4. Compare the performances of the two algorithms using the part of the data set that is reserved for testing, and report:
• The time needed for training as a function of the number of data points used for training.
• The prediction error (and standard deviation) as a function of the number of data points used for training.
5. Write a report describing the implemented algorithms and discussing the results and the learned lessons.

In [1]:
# if f(x) >= 0 then y = face
# else y = not face

#f(x) = w1*phi(x) + w2*phi2(x) + ..
# phi(x) = number of black pixels (example)

# intialize weights randomonly at first

# training condition is while less than 5 minutes or
# the weights aren't changing from image to image

#  update weights in training
 # if f(x) >=0 and y =face
 # or  f(x) <= 0 and y = not face
 # then do nothing

 # else if f(x) >= 0 and y = not face
 # then w1 = w1 -phi1 (x) 
 #      w2 = w2 - phi2(x)

# else if f(x) < 0 and y = face
 # then w1 = w1 + phi1 (x) 
 #      w2 = w2 + phi2(x)


#f = w1 * phi1(x) + w2 * phi2(x)



In [2]:
# Determine features - will need to obtain from image
# First step is get data from facedatatrain to use in this notebook
# Each face image is 70x60 (row x column) and each face begins and ends with an empty line (not including the 70x60).
# Then make a feature matrix of size 70*60, where each pixel is a feature

In [3]:
! pip install numpy

You should consider upgrading via the 'C:\Users\nafri\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [277]:
import sys
sys.path.insert(1, 'C:/Users/nafri/College/Senior Year/Spring Semester/Intro to AI/AI-FinalProject/Final Project/Berkeley_Example')
sys.path.insert(1, 'C:/Users/jzlyn/OneDrive/Desktop/CS440/AI-FinalProject/Final Project/Berkeley_Example')
sys.path.insert(1, 'C:/Users/chack/OneDrive/Documents/Intro to AI/Final Project/AI-FinalProject/Final Project/Berkeley_Example')

In [276]:
from Berkeley_Example import samples
from Berkeley_Example.perceptron import PerceptronClassifier
import numpy as np
import math

In [6]:
def sigmoid(x):
    try:
        sig =1/(1 + math.exp(-x))
    except:
        sig = 0   
    return sig

In [7]:
def update_weights(image,weights,over_estimate):
    for i in range(len(weights)):
        for j in range(len(weights[0])):
            if over_estimate:
                weights[i][j] -= image[i][j]
            else:
                weights[i][j] += image[i][j]

In [8]:
def face_train(n,percent):
    face_train_images = samples.loadDataFile('data/facedata/facedatatrain', n, 60, 70)
    face_train_labels = samples.loadLabelsFile('data/facedata/facedatatrainlabels', n)
    
    size = n * percent

    weights = np.random.uniform(-1, 1, size=(70, 60))
    bias = np.random.uniform(-1, 1)
    f = np.zeros(int(size)).astype(float)

    image_lst = []

    rand_samples = np.random.choice(n, int(size), replace=False)
    
    for i in rand_samples:
        matrix = face_train_images[i].getPixels()
        for r in range(len(matrix)):
            for c in range(len(matrix[0])):
                matrix[r][c] = samples.IntegerConversionFunction(matrix[r][c])
        image_lst.append(matrix)
            

    for i in range(len(image_lst)):
        f[i] = bias
        image = image_lst[i]
        for r in range(len(image)):
            for c in range(len(image[0])):
                f[i] += weights[r][c] * image[r][c]

        if f[i] >= 0 and face_train_labels[rand_samples[i]] == 0:
            update_weights(image,weights,True)
            bias -= 1
        elif f[i] < 0 and face_train_labels[rand_samples[i]] == 1:
            update_weights(image,weights,False)
            bias += 1

    return (weights, bias)  

In [9]:
def face_validate(n,percent,weights,bias):
    face_valid_images = samples.loadDataFile('data/facedata/facedatavalidation', n, 60, 70)
    face_valid_labels = samples.loadLabelsFile('data/facedata/facedatavalidationlabels', n)

    size = n * percent
 
    f = np.zeros(int(size)).astype(float)

    image_lst = []

    rand_samples = np.random.choice(n, int(size), replace=False)
    
    for i in rand_samples:
        matrix = face_valid_images[i].getPixels()
        for r in range(len(matrix)):
            for c in range(len(matrix[0])):
                matrix[r][c] = samples.IntegerConversionFunction(matrix[r][c])
        image_lst.append(matrix)

    for i in range(len(image_lst)):
        f[i] = bias
        image = image_lst[i]
        for r in range(len(image)):
            for c in range(len(image[0])):
                f[i] += weights[r][c] * image[r][c]

    num_correct = 0
    for i in range(len(f)):
        if (f[i] >= 0 and face_valid_labels[rand_samples[i]] == 1) or (f[i] < 0 and face_valid_labels[rand_samples[i]] == 0):
            num_correct += 1

    result = num_correct / size

    return result

In [10]:
TRAIN_SIZE = 451
VALID_SIZE = 301
epochs = 5
percentage = np.arange(0.1,1.1,.1)

for percent in percentage:
    accuracy = []
    for i in range(epochs):
        weights, bias = face_train(TRAIN_SIZE,percent) # return weights and bias used for validation
        result = face_validate(VALID_SIZE,percent,weights,bias)
        accuracy.append(result)

    # Optional - Save to file later    
    print(f'Percent of Training Data Used = {"{:.2%}".format(percent)}')
    print(accuracy)
    print(f'Mean: {np.mean(accuracy)}')
    print(f'Std: {np.std(accuracy)}')
    print()

Percent of Training Data Used = 10.00%
[0.4983388704318937, 0.6312292358803986, 0.7641196013289037, 0.7308970099667774, 0.4983388704318937]
Mean: 0.6245847176079733
Std: 0.11197541226812438

Percent of Training Data Used = 20.00%
[0.7475083056478405, 0.6146179401993355, 0.7308970099667774, 0.6976744186046512, 0.7641196013289037]
Mean: 0.7109634551495015
Std: 0.052948097842223374

Percent of Training Data Used = 30.00%
[0.6423034330011074, 0.7419712070874861, 0.6866002214839423, 0.7530454042081948, 0.7419712070874861]
Mean: 0.7131782945736433
Std: 0.042372373133353236

Percent of Training Data Used = 40.00%
[0.813953488372093, 0.7308970099667774, 0.8222591362126246, 0.7724252491694352, 0.8222591362126246]
Mean: 0.792358803986711
Std: 0.03585885904472243

Percent of Training Data Used = 50.00%
[0.6578073089700996, 0.6777408637873754, 0.7441860465116279, 0.8172757475083057, 0.7973421926910299]
Mean: 0.7388704318936877
Std: 0.06309144170349087

Percent of Training Data Used = 60.00%
[0.808

In [104]:
def train_digit(n,percent):
    #np.random.seed(405)
    # Read input from training digit file
    images = samples.loadDataFile('data/digitdata/trainingimages', n, 28, 28)
    labels = samples.loadLabelsFile('data/digitdata/traininglabels', n)

    size = int(n * percent)
    weights = np.random.randint(-5, 5, size=(10, 28, 28))
    bias = np.random.randint(-5, 5, size=10)
    
    # Randomly select images based on size
    rand_samples = np.random.choice(n, size, replace=False)

    # List to all converted images
    image_lst = []

    # Convert image into integers
    for i in rand_samples:
        image = images[i].getPixels()
        for r in range(28):
            for c in range(28):
               image[r][c] = samples.IntegerConversionFunction(image[r][c])
        image_lst.append(image)

    # Multiple iterations to make algo more accurate b/c there's so much data
    for _ in range(5):
        # Get predicted score for each digit
        # Keep track of the digit with the highest score 
        for i in range(len(image_lst)):
            image = image_lst[i]
            predict = 0
            max_predict = np.NINF
            arg_max = 0
            for d in range(10):
                for r in range(28):
                    for c in range(28):
                        predict += weights[d][r][c] * image[r][c]
                if predict > max_predict:
                    arg_max = d
                    max_predict = predict
                    
            # For digit with highest score, compare that to the actual score
            # If not equal update weights 
            index = rand_samples[i]
            if arg_max != labels[index]:
                bias[arg_max] -= 1
                bias[labels[index]] += 1
                for r in range(28):
                    for c in range(28):
                        weights[arg_max][r][c] -= image[r][c]
                        weights[labels[index]][r][c] += image[r][c]
                
    
    return (weights,bias)

In [105]:
def validate_digit(n,percent,weights,bias):
    # Read input from validation digit file
    images = samples.loadDataFile('data/digitdata/validationimages', n, 28, 28)
    labels = samples.loadLabelsFile('data/digitdata/validationlabels', n)

    size = int(n * percent)

    # Randomly select images based on size
    rand_samples = np.random.choice(n, size, replace=False)

    # List to all converted images
    image_lst = []

    # Convert image into integers
    for i in rand_samples:
        image = images[i].getPixels()
        for r in range(28):
            for c in range(28):
               image[r][c] = samples.IntegerConversionFunction(image[r][c])
        image_lst.append(image)

    correct_predicts = 0
    # Don't need to iterate 5 times, not training so just let validation run
    for i in range(len(image_lst)):
        image = image_lst[i]
        predict = 0
        max_predict = np.NINF
        arg_max = 0
        for d in range(10):
            for r in range(28):
                for c in range(28):
                    predict += weights[d][r][c] * image[r][c]
            if predict > max_predict:
                arg_max = d
                max_predict = predict
                
        index = rand_samples[i]
        if arg_max == labels[index]:
            correct_predicts += 1
    
    return correct_predicts / size

In [106]:
TRAIN_SIZE = 5000
VALID_SIZE = 1000
epochs = 5

percent = 1
accuracy = []
for i in range(epochs):
    weights, bias = train_digit(TRAIN_SIZE,percent) # return weights and bias used for validation
    result = validate_digit(VALID_SIZE,percent,weights,bias)
    accuracy.append(result)

# Optional - Save to file later    
print(f'Percent of Training Data Used = {"{:.2%}".format(percent)}')
print(accuracy)
print(f'Mean: {np.mean(accuracy)}')
print(f'Std: {np.std(accuracy)}')
print()

Percent of Training Data Used = 100.00%
[0.832, 0.786, 0.828, 0.819, 0.822]
Mean: 0.8173999999999999
Std: 0.01634135857265237



In [198]:
def cost(predict,actual,weights,n,d): # actual = labels
  J = 0
  reg = 0
  lam = 0.01
  for i in range(n):
    J += actual*math.log(predict)+(1-actual)*math.log(1-predict) # ln for log
  J = J/n * -1
  reg = np.sum(weights*weights) * lam/(2*n) # asumming this array element stuff works
  J = J + reg

In [275]:
num_imag = 25
conv_imag = []

face_train_imgs = samples.loadDataFile('data/facedata/facedatatrain', num_imag, 60, 70)
face_train_labs = samples.loadLabelsFile('data/facedata/facedatatrainlabels', num_imag)

for i in range(num_imag):
    matrix = face_train_imgs[i].getPixels()
    for r in range(len(matrix)):
        for c in range(len(matrix[0])):
            matrix[r][c] = samples.IntegerConversionFunction(matrix[r][c])
    conv_imag.append(matrix)
    
W1 = np.random.randn(1800, 4200) * np.sqrt(2.0 / (4200 + 1800))
b1 = np.ones(1800)
W2 = np.random.randn(1, 1800) * np.sqrt(2.0 / (1800 + 1))
b2 = 1.0
#maybe intialize to zero the grad if necessary
# make sure the gradients are computed correctly like in slides
for epoch in range(5):
    for i in range(len(conv_imag)): 
        a1 = np.array([element for row in conv_imag[i] for element in row])# 4200 x1
        #forward prop
        z2 = np.array([np.dot(a1, W1[hidNode,:]) + b1[hidNode] for hidNode in range(W1.shape[0])])
        a2 =np.array(list(map(sigmoid,z2))) #1800x1
        z3 = (np.dot(a2, W2.T) + b2)[0]
        a3 = sigmoid(z3) # 1x1
        delta3 = a3 - face_train_labs[i] # 1x1
        delta2 = (W2.T * delta3) * (a2 *(1-a2)).reshape(1800,1)# 1800x1 , W2: 1x1800
        delta_W1 = delta2 *a1.T# grad(layer1)
        delta_W2 = delta3 * a2.T  # grad(layer 2)
        D_W1 = delta_W1/ len(conv_imag) + 0.2 * W1 # lambda =0.2
        D_W2 = delta_W2/ len(conv_imag) + 0.2 * W2
        W1 -= 0.01 * D_W1# alpha = 0.01
        print("W1",W1)
        W2 -= 0.01 * D_W2
        print("W2",W2)

W1 [[-0.01192678  0.01373239  0.00778345 ... -0.01248003 -0.02203159
   0.00747101]
 [-0.02167441 -0.01856421  0.00987237 ...  0.00043138 -0.0096255
   0.02190097]
 [ 0.02352308  0.01455868 -0.01922866 ...  0.01861376 -0.01476886
  -0.0093315 ]
 ...
 [-0.01507944 -0.00810482 -0.01514236 ... -0.01478769 -0.0021853
   0.0150928 ]
 [ 0.01856157 -0.02061632 -0.01136991 ...  0.0049122  -0.00821909
   0.00792676]
 [-0.00049265 -0.00196753 -0.02846736 ...  0.00678764  0.0506466
   0.03934758]]
W2 [[-0.01727665  0.04192585 -0.01096496 ... -0.02678529  0.02379075
   0.0480312 ]]
W1 [[-0.01190293  0.01370493  0.00776789 ... -0.01245507 -0.02198753
   0.00745607]
 [-0.02163106 -0.01852708  0.00985262 ...  0.00043052 -0.00960625
   0.02185717]
 [ 0.02347604  0.01452957 -0.0191902  ...  0.01857653 -0.01473933
  -0.00931284]
 ...
 [-0.01504928 -0.00808861 -0.01511208 ... -0.01475811 -0.00218093
   0.01506261]
 [ 0.01852445 -0.02057509 -0.01134717 ...  0.00490238 -0.00820266
   0.00791091]
 [-0.00049